In [1]:
import pandas as pd

In [63]:
df = pd.read_csv("../DataSets/titanic.csv")

In [8]:
# Remove specific features

In [64]:
df = df.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)

In [47]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S,cluster_group
0,0,3,1,22.0,1,0,7.2500,False,False,True,0.0
1,1,1,0,38.0,1,0,71.2833,True,False,False,1.0
2,1,3,0,26.0,0,0,7.9250,False,False,True,0.0
3,1,1,0,35.0,1,0,53.1000,False,False,True,1.0
4,0,3,1,35.0,0,0,8.0500,False,False,True,0.0


In [ ]:
# convert male, female to 1,0

In [22]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [65]:
df['Sex'] = le.fit_transform(df['Sex'].astype(str))

In [27]:
# Enmbarked is one hot encoded

In [66]:
df = pd.get_dummies(df, columns=['Embarked'])

In [ ]:
# check null values

In [29]:
df.isnull().sum()

Survived        0
Pclass          0
Sex             0
Age           177
SibSp           0
Parch           0
Fare            0
Embarked_C      0
Embarked_Q      0
Embarked_S      0
dtype: int64

In [30]:
df[df.isnull().any(axis=1)]

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
5,0,3,1,NaN,0,0,8.4583,False,True,False
17,1,2,1,NaN,0,0,13.0000,False,False,True
19,1,3,0,NaN,0,0,7.2250,True,False,False
26,0,3,1,NaN,0,0,7.2250,True,False,False
28,1,3,0,NaN,0,0,7.8792,False,True,False
...,...,...,...,...,...,...,...,...,...,...
859,0,3,1,NaN,0,0,7.2292,True,False,False
863,0,3,0,NaN,8,2,69.5500,False,False,True
868,0,3,1,NaN,0,0,9.5000,False,False,True
878,0,3,1,NaN,0,0,7.8958,False,False,True


In [67]:
df = df.dropna()

In [32]:
#Mean shift clustering

In [33]:
from sklearn.cluster import MeanShift

In [40]:
analyzer = MeanShift(bandwidth=30)

In [41]:
analyzer.fit(df)

MeanShift(bandwidth=30)

In [36]:
# guassian function bandwidth need not be guessed, it can be estimated

In [38]:
from sklearn.cluster import estimate_bandwidth

In [39]:
estimate_bandwidth(df)

30.44675914497196

In [42]:
labels = analyzer.labels_

In [55]:
# 5 clusters were generated by algo

In [44]:
import numpy as np
np.unique(labels)

array([0, 1, 2, 3, 4], dtype=int64)

In [48]:
# add cluster to each row of the data frame, indicating which cluster passenger belongs to

In [46]:
df['cluster_group'] = np.nan
length = len(df)

for i in range(length):
    df.iloc[i,df.columns.get_loc('cluster_group')] = labels[i]

In [49]:
df_data = df.groupby(['cluster_group']).mean()

In [53]:
df_data['Counts'] = pd.Series(df.groupby(['cluster_group']).size())

In [54]:
df_data

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S,Counts
cluster_group,,,,,,,,,,,
0.0,0.336918,2.525090,0.679211,28.256720,0.439068,0.370968,15.434139,0.121864,0.046595,0.831541,558
1.0,0.611111,1.296296,0.527778,36.148148,0.814815,0.500000,65.622688,0.333333,0.018519,0.629630,108
2.0,0.733333,1.000000,0.366667,32.430667,0.600000,0.866667,131.183883,0.500000,0.000000,0.500000,30
3.0,0.733333,1.000000,0.266667,30.333333,1.000000,1.333333,239.991940,0.533333,0.000000,0.466667,15
4.0,1.000000,1.000000,0.666667,35.333333,0.000000,0.333333,512.329200,1.000000,0.000000,0.000000,3


In [56]:
# cluster 0 survival rate is only 33%
# cluster 4 survival rate is 100%

In [57]:
# cluster 4 has high fare where as cluser 0 as small dare

In [76]:
# K Means

In [58]:
from sklearn.cluster import KMeans

In [60]:
km = KMeans(n_clusters=4)

In [68]:
km.fit(df)

KMeans(n_clusters=4)

In [71]:
lab = km.labels_

In [74]:
df['cluster_group'] = np.nan
for i in range(len(df)):
    df.iloc[i:df.columns.get_loc('cluster_group')] = lab[i]

C:\Users\ST\AppData\Local\Temp\ipykernel_23872\3561748095.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df.iloc[i:df.columns.get_loc('cluster_group')] = lab[i]
C:\Users\ST\AppData\Local\Temp\ipykernel_23872\3561748095.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df.iloc[i:df.columns.get_loc('cluster_group')] = lab[i]
C:\Users\ST\AppData\Local\Temp\ipykernel_23872\3561748095.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df.iloc[i:df.columns.get_loc('cluste

In [75]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S,cluster_group
0,0,0,0,0.0,0,0,0.0,0,0,0,0.0
1,3,3,3,3.0,3,3,3.0,3,3,3,3.0
2,0,0,0,0.0,0,0,0.0,0,0,0,0.0
3,3,3,3,3.0,3,3,3.0,3,3,3,3.0
4,0,0,0,0.0,0,0,0.0,0,0,0,0.0
